In [3]:
!pip install rusenttokenize

In [29]:
import pandas as pd
from ast import literal_eval
from ru_sent_tokenize import ru_sent_tokenize as st
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy
import torch
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
from pprint import pprint

In [7]:
df = pd.read_csv('additional_data/parsed_corpus_changed.csv')
df.target = df.target.apply(literal_eval)
df.source = df.source.apply(literal_eval)

In [9]:
df

,source,target,link
0,[В июне в этнографическом музее «Торум Маа» го...,[Лӯпта э̄тпост Ханты-Мансийск ӯст «То̄рум Ма...,https://khanty-yasang.ru//luima-seripos/no-14-...
1,"[Демид Чамалтынов, 9 лет, – Я живу в Ханты-Ман...","[Демид Чамалтынов, 9 та̄л, – «Та̄ксар ма̄хум» ...",https://khanty-yasang.ru//luima-seripos/no-16-...
2,[В Берёзовском районе в семи километрах ниже о...,[Ха̄льӯс районт Кульпас па̄вылныл ло̄ӈхаль са...,https://khanty-yasang.ru//luima-seripos/no-17-...
3,[Я хочу написать про своих детей. Этот год для...,[Ам амки на̄врама̄гум урыл хансуӈкв таӈхе̄гум....,https://khanty-yasang.ru//luima-seripos/no-13-...
4,[Каждое лето Ханты-Мансийский этнографический ...,[Ка̄сыӈ туи пора Ханты-Мансийск ӯст о̄лнэ на̄...,https://khanty-yasang.ru//luima-seripos/no-17-...
...,...,...,...
3746,[С хантыйской мастерицей Надеждой Алексеевной ...,[Надежда Алексеевна Гришкина ты та̄л атпан нуп...,https://khanty-yasang.ru//luima-seripos/no-24-...
3747,[Для жителей сельского поселения села Саранпау...,[Ма̄н округувт са̄в са̄лыт туп тит ма̄т о̄ньща...,https://khanty-yasang.ru//luima-seripos/no-5-1...
3748,[Альбина Волосовская уже много лет живет и раб...,[Ты хурит Октябрьский па̄вылт о̄лнэ ма̄ньщи нэ...,https://khanty-yasang.ru//luima-seripos/no-14-...
3749,[В Ханты-Мансийске в июне месяце состоялись «I...,[Ха̄льӯст ма̄ньлат ма̄ньщи хум Михаил Яркин о...,https://khanty-yasang.ru//luima-seripos/no-13-...


In [11]:
for paragraph in [st(el) for el in df.target.loc[230]]:
    for sent in paragraph[:2]:
        print(sent, end='\n--------------------\n')

Ханты пыг Виталий Сигильетов хӯрум та̄л Венгрият о̄лы, тав 2016 та̄лт Югорский университет а̄стламе юи-па̄лт тув минас.
--------------------
Ма̄ньлат пыг тыт туризм тэ̄лат тотнэ ос ма̄-вит ӯргалан хо̄тпаг ханищтахтас.
--------------------
Виталий самын патум ма̄тэ̄т хантыт мощ мо̄т щирыл потыртэ̄гыт, ла̄тӈаныл «ваховскииг» ла̄ваве.
--------------------
Та̄н Нижневартовский районт о̄лэ̄гыт ос ты ла̄тӈыл потыртан хо̄тпа мощща хультыс.
--------------------


In [17]:
model = SentenceTransformer('C:/Users/Vladimir/PycharmProjects/perevod/models/checkpoint-39240', device='cuda')

In [19]:
df.source = df.source.apply(lambda x: [st(el) for el in x])
df.target = df.target.apply(lambda x: [st(el) for el in x])

In [21]:
# удаление пустых абзацев
df.target = df.target.apply(lambda x: [el for el in x if len(el) > 0])
df.source = df.source.apply(lambda x: [el for el in x if len(el) > 0])

In [23]:
# удаление пустых предложений
df.target = df.target.apply(lambda x: [[c for c in el if len(c) > 0] for el in x])
df.source = df.source.apply(lambda x: [[c for c in el if len(c) > 0] for el in x])

In [33]:
temp = df

rus_mansi_all = []
mansi_rus_all = []

for _, row in tqdm(temp.iterrows(), total=len(temp)):
    mansi_sents = sum(row.target, [])
    rus_sents = sum(row.source, [])
    mansi_embs = model.encode(mansi_sents)
    rus_embs = model.encode(rus_sents)
    cos_sim = cosine_similarity(rus_embs, mansi_embs) * 100

    rus_top5_mean = np.mean(np.sort(cos_sim, axis=1)[:, -5:], axis=1)
    mansi_top5_mean = np.mean(np.sort(cos_sim, axis=0)[-5:, :], axis=0)

    cos_sim_norm_rows = cos_sim / rus_top5_mean[:, np.newaxis]
    cos_sim_norm = cos_sim_norm_rows / mansi_top5_mean[np.newaxis, :]
    
    rus_to_mansi_indices = np.argmax(cos_sim, axis=1)
    mansi_to_rus_indices = np.argmax(cos_sim, axis=0)
    
    rus_to_mansi = [
        {
            'id_rus': i,
            'id_mansi': idx,
            'rus': rus_sents[i],
            'mansi': mansi_sents[idx],
            'score': cos_sim[i, idx]
        }
        for i, idx in enumerate(rus_to_mansi_indices)
    ]
    rus_to_mansi = sorted(rus_to_mansi, key=lambda x: x['id_rus'])
    
    mansi_to_rus = [
        {
            'id_rus': idx,
            'id_mansi': i,
            'mansi': mansi_sents[i],
            'rus': rus_sents[idx],
            'score': cos_sim[idx, i]
        }
        for i, idx in enumerate(mansi_to_rus_indices)
    ]
    mansi_to_rus = sorted(mansi_to_rus, key=lambda x: x['id_mansi'])

    
    
    rus_mansi_all.append(deepcopy(rus_to_mansi))
    mansi_rus_all.append(deepcopy(mansi_to_rus))



  0%|          | 0/3751 [00:00<?, ?it/s]

In [35]:
cos_sim.shape

(91, 83)

In [39]:
pd.set_option('display.max_colwidth', None)

In [41]:
pd.merge(
    pd.DataFrame(rus_mansi_all[0]).rename(
        columns={
            'rus': 'rus_from_rus',
            'mansi': 'mansi_from_rus',
            'id_mansi': 'id_mansi_from_rus',
            'score': 'score_from_rus'
        }
    ),
    pd.DataFrame(mansi_rus_all[0]).rename(
        columns={
            'rus': 'rus_from_mansi',
            'mansi': 'mansi_from_mansi',
            'id_mansi': 'id_mansi_from_mansi',
            'score': 'score_from_mansi'
        }
    ),
    on='id_rus',
    how='left'
) \
    .groupby('id_rus', as_index=False) \
    .agg(
        id_rus=('id_rus', 'max'),
        # rus
        id_mansi_from_rus=('id_mansi_from_rus', 'max'),
        rus_from_rus=('rus_from_rus', 'max'),
        mansi_from_rus=('mansi_from_rus', 'max'),
        score_from_rus=('score_from_rus', 'max'),
        # mansi
        id_mansi_from_mansi=('id_mansi_from_mansi', lambda x: [int(el) for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        rus_from_mansi=('rus_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        mansi_from_mansi=('mansi_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        score_from_mansi=('score_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None)
    )

,id_rus,id_mansi_from_rus,rus_from_rus,mansi_from_rus,score_from_rus,id_mansi_from_mansi,rus_from_mansi,mansi_from_mansi,score_from_mansi
0,0,7,"В июне в этнографическом музее «Торум Маа» города Ханты-Мансийска дети посещали лагерь «Таксар махум – Крепкие люди», где приобрели ценные навыки для жизни в природе – научились добывать огонь, собирать рюкзак с необходимыми предметами для похода в лес, стрелять из лука.","Ханты ос ма̄ньщи ма̄хум на̄враманыл хумус ёнгалтыяныл, манхурип ёнгилыт ва̄руӈкве ха̄сэ̄гыт, о̄с суссӯв, мощ ёнгасасӯв.",51.982288,[0],"[В июне в этнографическом музее «Торум Маа» города Ханты-Мансийска дети посещали лагерь «Таксар махум – Крепкие люди», где приобрели ценные навыки для жизни в природе – научились добывать огонь, собирать рюкзак с необходимыми предметами для похода в лес, стрелять из лука.]",[Лӯпта э̄тпост Ханты-Мансийск ӯст «То̄рум Маа» музейт на̄врамыт ат хо̄тал ханищтахтасыт.],[49.91943359375]
1,1,41,Ребята рассказали:,На̄врамыт ёт Светлана Ромбандеева потыртас,22.126839,None,None,None,None
2,2,18,"Ваня Зайчиков, 12 лет","Ваня Зайчиков, 12 лет",99.999985,[18],"[Ваня Зайчиков, 12 лет]","[Ваня Зайчиков, 12 лет]",[99.99998474121094]
3,3,19,"– Я приехал из Новосибирска, живу там и учусь в 5-й гимназии.","– Ты ма̄н ам Новосибирск ӯсныл ёхтысум, тот 5 гимназият ханищтахтэ̄гум.",86.012177,[19],"[– Я приехал из Новосибирска, живу там и учусь в 5-й гимназии.]","[– Ты ма̄н ам Новосибирск ӯсныл ёхтысум, тот 5 гимназият ханищтахтэ̄гум.]",[86.01217651367188]
4,4,20,"Дедушке надо было решить дела в Ханты-Мансийске, я с ним приехал.","о̄пам Ханты-Мансийск ӯст са̄в тэ̄лат о̄ньщи, тыг ва̄тихал ёхталы, ам о̄с тав ётэ ӯнтылматсум, тыт ты минэ̄гум.",46.562267,None,None,None,None
5,5,13,"Он увидел объявление о лагере «Таксар махум», и я решил, ну попробую, и мне очень понравилось.","Музейн омам ос а̄тям ёт ёхталасум кос, ты ма̄т ё̄мыгтасум, ос лагерь урыл омам ётыл ущ хӯнтамластэ, ам намум тыг хансыстэ.",48.198181,"[1, 21]","[Он увидел объявление о лагере «Таксар махум», и я решил, ну попробую, и мне очень понравилось., Он увидел объявление о лагере «Таксар махум», и я решил, ну попробую, и мне очень понравилось.]","[Ты тэ̄ла ка̄сыӈ туи ва̄рыглаве, ма̄ньщи ос рущ щирыл намаявес «Та̄ксар ма̄хум – Крепкие люди»., «Та̄ксар ма̄хум» лагерин о̄с тав а̄нум хансыстэ.]","[44.27549362182617, 39.26034927368164]"
6,6,8,Больше всего мне понравилось делать луки.,"Ам сака таӈхе̄гум ма̄ньщи па̄вылн ялуӈкве, тот ма̄ньха̄пыл товнувум, сунсуӈкве, сосса мир хумус о̄лэ̄гыт.",32.963978,None,None,None,None
7,7,31,В своём городе я хожу на дзюдо и плавание.,"Та̄рвитыӈте̄пыл, ам финиш мус та ёхтахтасум.",34.956886,None,None,None,None
8,8,20,"В Ханты-Мансийске часто бываю, город очень красивый, много деревьев, зелени.","о̄пам Ханты-Мансийск ӯст са̄в тэ̄лат о̄ньщи, тыг ва̄тихал ёхталы, ам о̄с тав ётэ ӯнтылматсум, тыт ты минэ̄гум.",63.727867,[20],"[В Ханты-Мансийске часто бываю, город очень красивый, много деревьев, зелени.]","[о̄пам Ханты-Мансийск ӯст са̄в тэ̄лат о̄ньщи, тыг ва̄тихал ёхталы, ам о̄с тав ётэ ӯнтылматсум, тыт ты минэ̄гум.]",[63.727867126464844]
9,9,3,"Аня Прохоренко, 12 лет:","Аня Прохоренко, 12 та̄л",87.849266,[3],"[Аня Прохоренко, 12 лет:]","[Аня Прохоренко, 12 та̄л]",[87.8492660522461]


In [47]:
pd.DataFrame(rus_mansi_all[0])#.query('score > 50')

,id_rus,id_mansi,rus,mansi,score
0,0,7,"В июне в этнографическом музее «Торум Маа» города Ханты-Мансийска дети посещали лагерь «Таксар махум – Крепкие люди», где приобрели ценные навыки для жизни в природе – научились добывать огонь, собирать рюкзак с необходимыми предметами для похода в лес, стрелять из лука.","Ханты ос ма̄ньщи ма̄хум на̄враманыл хумус ёнгалтыяныл, манхурип ёнгилыт ва̄руӈкве ха̄сэ̄гыт, о̄с суссӯв, мощ ёнгасасӯв.",51.982288
1,1,41,Ребята рассказали:,На̄врамыт ёт Светлана Ромбандеева потыртас,22.126839
2,2,18,"Ваня Зайчиков, 12 лет","Ваня Зайчиков, 12 лет",99.999985
3,3,19,"– Я приехал из Новосибирска, живу там и учусь в 5-й гимназии.","– Ты ма̄н ам Новосибирск ӯсныл ёхтысум, тот 5 гимназият ханищтахтэ̄гум.",86.012177
4,4,20,"Дедушке надо было решить дела в Ханты-Мансийске, я с ним приехал.","о̄пам Ханты-Мансийск ӯст са̄в тэ̄лат о̄ньщи, тыг ва̄тихал ёхталы, ам о̄с тав ётэ ӯнтылматсум, тыт ты минэ̄гум.",46.562267
5,5,13,"Он увидел объявление о лагере «Таксар махум», и я решил, ну попробую, и мне очень понравилось.","Музейн омам ос а̄тям ёт ёхталасум кос, ты ма̄т ё̄мыгтасум, ос лагерь урыл омам ётыл ущ хӯнтамластэ, ам намум тыг хансыстэ.",48.198181
6,6,8,Больше всего мне понравилось делать луки.,"Ам сака таӈхе̄гум ма̄ньщи па̄вылн ялуӈкве, тот ма̄ньха̄пыл товнувум, сунсуӈкве, сосса мир хумус о̄лэ̄гыт.",32.963978
7,7,31,В своём городе я хожу на дзюдо и плавание.,"Та̄рвитыӈте̄пыл, ам финиш мус та ёхтахтасум.",34.956886
8,8,20,"В Ханты-Мансийске часто бываю, город очень красивый, много деревьев, зелени.","о̄пам Ханты-Мансийск ӯст са̄в тэ̄лат о̄ньщи, тыг ва̄тихал ёхталы, ам о̄с тав ётэ ӯнтылматсум, тыт ты минэ̄гум.",63.727867
9,9,3,"Аня Прохоренко, 12 лет:","Аня Прохоренко, 12 та̄л",87.849266


In [63]:
all_dict = {
    'rus': [],
    'mansi': [],
    'score': []
}
for i in tqdm(range(len(mansi_rus_all))):
    rm = pd.DataFrame(rus_mansi_all[i])
    mr = pd.DataFrame(mansi_rus_all[i])
    a = pd.merge(rm, mr, on=['id_rus', 'id_mansi'], how='inner')
    all_dict['rus'].extend(a['rus_x'].tolist())
    all_dict['mansi'].extend(a['mansi_x'].tolist())
    all_dict['score'].extend(a['score_x'].tolist())

full_govno = pd.DataFrame(all_dict)


  0%|          | 0/3751 [00:00<?, ?it/s]

In [103]:
full_govno.query('score >= 30 & score < 95').sort_values(by='score', ascending=True).tail(10)

,rus,mansi,score
28851,"Яна Ларионова, 12 лет:","Яна Ларионова, 12 та̄л:",94.603905
14576,"Диплом «Лучший европейский фильм для детей» получил фильм «Сёстры», режиссёры: Силье Саломонсен и Арильд Остин Оммундсен, Норвегия.","«Лучший европейский фильм для детей» дипломыл «Сёстры» кина майвес, режиссёрыг Силье Саломонсен ос Арильд Остин Оммундсен Норвегияныл ёхталасы̄г.",94.616035
42786,Сайнахов Игнатий Афанасьевич,Игнат Афанасьевич Сайнахов,94.637314
25528,Даша Дунаева,Даша ДУНАЕВА,94.667854
9927,В номинации «Педагогические инновации»:,«Педагогические инновации» нампа номинации:,94.742599
473,"Александр Орлов, 12 лет","Александр Орлов, 12 та̄л",94.777740
2383,"Дима Шишкин, 12 лет","Дима Шишкин, 12 та̄л",94.796890
39191,Хозумов Пётр Степанович,Пётр Степанович Хозумов,94.800117
11749,Тюменская областная Дума,Тюменский область Дума,94.920883
2315,"Дима Карпов, 11 лет","Дима Карпов, 11 та̄л",94.977570


In [111]:
full_govno.query('score >= 30 & score < 95').to_csv('additional_data/44k_parse_Labse_data.csv', index=False)